In [2]:
import pandas as pd
import numpy as np

In [112]:
df = pd.read_csv('tidy_2.csv')

## 새로운 칼럼 만들기

### 1) 대상인원 칼럼 만들기

In [4]:
df['대상인원'] = df['대상인원'].copy().str.extract(r'[\s|외|총|등](\d+)\s?[명|인]?')
# 대상인원이 없는 경우 운전기사 회식, 격려금 지급 등 맛집과 상관없는 경우가 많으니 삭제
df = df.dropna(axis=0, subset=['대상인원'], how='all')
df['대상인원'] = df['대상인원'].copy().astype(int)


### 2) 년도별로 그룹화하기

In [113]:
df_2016 = df[df['year'] == 2016]
df_2017 = df[df['year'] == 2017]
df_2018 = df[df['year'] == 2018]

dfs = [df, df_2016, df_2017, df_2018]
df_2016.shape, df_2017.shape, df_2018.shape

((3780, 14), (13386, 14), (7857, 14))

### 3) 집행지명으로 그룹화하기

In [27]:
def group_place(df):
    df_group = df.groupby('집행지명')\
            ['집행지명', 'cnt', '집행금액', '대상인원']\
            .sum().reset_index()
    return df_group

df_2016_group = group_place(df_2016)
df_2017_group = group_place(df_2017)
df_2018_group = group_place(df_2018)
df_group = group_place(df)

df_groups = [df_group, df_2016_group, df_2017_group, df_2018_group]
df_group.shape, df_2016_group.shape, df_2017_group.shape, df_2018_group.shape

((3752, 4), (1090, 4), (2403, 4), (1749, 4))

In [36]:
# 회당, 인당 금액을 구해보자. 
def price_per_num_people(df_group):
    df_group['회당집행금액'] = df_group['집행금액'] / df_group['cnt']
    df_group['인당집행금액'] = df_group['집행금액'] / df_group['대상인원']
    df_group['회당명수'] = df_group['대상인원'] / df_group['cnt']

for df_group in df_groups:
    df_group = price_per_num_people(df_group)

df_groups[0].head()

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
0,(월)월향,1,34000.0,2,34000.000000,17000.000000,2.000000
1,(주)강가,7,857000.0,30,122428.571429,28566.666667,4.285714
2,(주)광화문아띠,1,40000.0,2,40000.000000,20000.000000,2.000000
3,(주)광화문아띠명동,1,455500.0,40,455500.000000,11387.500000,40.000000
4,(주)교보문고,2,160000.0,14,80000.000000,11428.571429,7.000000


# 서울시 진짜 맛집 프로젝트

## 카테고리별 맛집

### 1. 가장 자주 간 식당 (전체 공무원)

In [48]:
def by_cnt(df_group):
    df_group = df_group.sort_values(by='cnt', ascending=False)
    df_group['회당명수'] = round(df_group['회당명수']).astype(int)
    df_group['회당집행금액'] = round(df_group['회당집행금액']).astype(int)
    df_group['인당집행금액'] = round(df_group['인당집행금액']).astype(int)
    df_group['집행금액'] = round(df_group['집행금액']).astype(int)
    return df_group

for i, df_group in enumerate(df_groups):
    df_groups[i] = by_cnt(df_group)

In [50]:
# 전체 공무원
df_groups[0].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620,4653,116834,17175,7
1006,동해일식,377,55124400,2248,146219,24522,6
1462,바닷가작은부엌,310,45153200,1920,145655,23517,6


In [51]:
# 2016년
df_groups[1].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
803,제일가든,74,17996020,794,243189,22665,11
301,라칸티나,68,12256400,359,180241,34140,5
860,참숯골,60,6074100,362,101235,16779,6


In [52]:
# 2017년
df_groups[2].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1896,참숯골,385,41967120,2577,109006,16285,7
2151,한가람,225,22636550,1357,100607,16681,6
633,동해수산,186,24744000,1006,133032,24596,5


In [53]:
# 2018년
df_groups[3].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1350,참숯골,238,31645400,1701,132964,18604,7
457,동해일식,188,30410000,1253,161755,24270,7
1356,창고43,126,14937300,807,118550,18510,6


### 2. 가장 돈을 많이 쓴 식당 

In [54]:
# 전체
df_groups[0].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620,4653,116834,17175,7
1006,동해일식,377,55124400,2248,146219,24522,6
1103,라칸티나,271,48128400,1812,177596,26561,7


In [59]:
# 2016년
df_groups[1].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
803,제일가든,74,17996020,794,243189,22665,11
301,라칸티나,68,12256400,359,180241,34140,5
761,이설당한과,1,10040000,251,10040000,40000,251


In [63]:
# 2017년
df_groups[2].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1896,참숯골,385,41967120,2577,109006,16285,7
633,동해수산,186,24744000,1006,133032,24596,5
1790,제일가든,158,23975000,1231,151741,19476,8


In [61]:
# 2018년
df_groups[3].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1350,참숯골,238,31645400,1701,132964,18604,7
457,동해일식,188,30410000,1253,161755,24270,7
553,만복림,117,22605700,985,193211,22950,8


### 3. 회당 집행금액이 가장 많은 식당
(30회 이상 방문, 전체 공무원)

In [64]:
for i, df_group in enumerate(df_groups):
    df_groups[i] = df_group[df_group['cnt']>30]

In [65]:
# 전체
df_groups[0].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
868,더미,107,32091000,1227,299916,26154,11
2913,진미족발,55,14689500,762,267082,19278,14
2004,송학,60,15229000,583,253817,26122,10


In [140]:
# 2016년
df_groups[1].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
803,제일가든,74,17996020,794,243189,22665,11
369,몽중헌,33,7339100,210,222397,34948,6
269,동해수산,41,7491000,286,182707,26192,7


In [66]:
# 2017년 
df_groups[2].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
546,더미,43,10678000,405,248326,26365,9
1540,오륙도,57,11692000,497,205123,23525,9
1113,삼원,41,7991000,329,194902,24289,8


In [68]:
# 2018년
df_groups[3].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
380,더미,48,16554000,655,344875,25273,14
1060,열빈,64,13733000,643,214578,21358,10
553,만복림,117,22605700,985,193211,22950,8


### 4. 인당 집행금액이 가장 많은 식당

In [69]:
# 전체
df_groups[0].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1154,루이,32,5866800,180,183338,32593,6
1372,몽중헌,62,12552500,403,202460,31148,6
797,달개비자연음식,47,9173950,310,195190,29593,7


In [70]:
# 2016
# 김영란법 생기기 전인가? 3만원이 넘는다. 
df_groups[1].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
369,몽중헌,33,7339100,210,222397,34948,6
301,라칸티나,68,12256400,359,180241,34140,5
270,동해일식,44,5707400,180,129714,31708,4


In [71]:
# 2017
df_groups[2].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
817,메이징에이,35,4752500,176,135786,27003,5
546,더미,43,10678000,405,248326,26365,9
557,더테이블,67,11078000,425,165343,26066,6


In [72]:
# 2018
df_groups[3].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1092,오륙도,31,5130500,198,165500,25912,6
128,강가,33,4131500,160,125197,25822,5
380,더미,48,16554000,655,344875,25273,14


### 5. 가장 많은 사람이 간 식당

In [75]:
# 전체
df_groups[0].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620,4653,116834,17175,7
2923,진주회관,115,6459000,2537,56165,2546,22
1006,동해일식,377,55124400,2248,146219,24522,6


In [76]:
# 2016
df_groups[1].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
803,제일가든,74,17996020,794,243189,22665,11
860,참숯골,60,6074100,362,101235,16779,6
301,라칸티나,68,12256400,359,180241,34140,5


In [77]:
# 2017
df_groups[2].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1896,참숯골,385,41967120,2577,109006,16285,7
1867,진주회관,62,3510000,2314,56613,1517,37
2151,한가람,225,22636550,1357,100607,16681,6


In [78]:
# 2018
df_groups[3].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1350,참숯골,238,31645400,1701,132964,18604,7
457,동해일식,188,30410000,1253,161755,24270,7
553,만복림,117,22605700,985,193211,22950,8


### 6. 단체손님이 많이 가는 식당
8명 이상의 단체손님이 가는 식당 중 가장 자주 가는 식당

In [94]:
# 전체
df_groups[0][df_groups[0]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2805,제일가든,232,41971020,2025,180910,20726,9
2343,열빈,191,38018500,1943,199050,19567,10
1735,산채향,162,26468000,1255,163383,21090,8


In [95]:
# 2016년
# 2016년엔 단체로 가지 않은 것일까, 데이터가 적은 것일까
df_groups[1][df_groups[1]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
803,제일가든,74,17996020,794,243189,22665,11


In [96]:
# 2017년
df_groups[2][df_groups[2]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1790,제일가든,158,23975000,1231,151741,19476,8
1493,열빈,104,19829500,1118,190668,17737,11
1103,산채향,93,16030000,747,172366,21459,8


In [142]:
# 2018년
# 2016, 2017년에 사랑받던 제일가든이 없어지고
# 열빈이 회식 대세인가보다
df_groups[3][df_groups[3]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
553,만복림,117,22605700,985,193211,22950,8
1060,열빈,64,13733000,643,214578,21358,10
380,더미,48,16554000,655,344875,25273,14


## 부서별로 살펴보기

In [117]:
# 부서별로 묶기
def make_divs(df):
    df_divs = df.groupby('1소속부서')['cnt', 'num', '집행금액']\
            .sum().reset_index()
    # 인당/회당 금액 내기
    df_divs['인당집행금액'] = df_divs['집행금액'] / df_divs['num']
    df_divs['회당집행금액'] = df_divs['집행금액'] / df_divs['cnt']
    df_divs['인당집행금액'] = round(df_divs['인당집행금액']).astype(int)
    df_divs['회당집행금액'] = round(df_divs['회당집행금액']).astype(int)
    return df_divs

df_divs=[]
for i, df in enumerate(dfs):
    df_divs.append(make_divs(df))

### 1. 가장 회식을 자주 한 부서

In [119]:
# 전체
df_divs[0].sort_values(by='cnt', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
25,행정국,2388,17071,297689495.0,17438,124661
16,시민소통기획관,2051,16715,276496282.0,16542,134810
20,정무부시장,1534,10530,228080150.0,21660,148683


In [123]:
# 2016
df_divs[1].sort_values(by='cnt', ascending=False).head(5)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
19,정무부시장,625,3891,90778450.0,23330,145246
23,행정국,395,2884,46919300.0,16269,118783
11,서울특별시장,288,3893,86490120.0,22217,300313
21,행정1부시장,269,1155,56968960.0,49324,211781
22,행정2부시장,259,1665,75801000.0,45526,292668


In [124]:
# 2017
df_divs[2].sort_values(by='cnt', ascending=False).head(5)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
25,행정국,1280,8992,154039840.0,17131,120344
16,시민소통기획관,1170,10317,164873359.0,15981,140917
6,대변인,871,5712,95435580.0,16708,109570
4,기획조정실,868,6337,101052048.0,15946,116419
5,기후환경본부,778,15021,70020101.0,4661,90000


In [125]:
# 2018
# 행정국 꾸준히 1위
df_divs[3].sort_values(by='cnt', ascending=False).head(5)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
24,행정국,692,4973,95344355.0,19172,137781
15,시민소통기획관,645,4702,81274853.0,17285,126008
3,기획조정실,561,3867,75619510.0,19555,134794
9,복지본부,524,3314,49630430.0,14976,94715
12,서울혁신기획관,448,2810,36315160.0,12924,81061


### 2. 가장 돈을 많이 쓴 부서

In [126]:
# 전체
df_divs[0].sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
25,행정국,2388,17071,297689495.0,17438,124661
16,시민소통기획관,2051,16715,276496282.0,16542,134810
24,행정2부시장,724,4923,231317260.0,46987,319499


In [127]:
# 2016
df_divs[1].sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
19,정무부시장,625,3891,90778450.0,23330,145246
11,서울특별시장,288,3893,86490120.0,22217,300313
22,행정2부시장,259,1665,75801000.0,45526,292668


In [128]:
# 2017
df_divs[2].sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
16,시민소통기획관,1170,10317,164873359.0,15981,140917
25,행정국,1280,8992,154039840.0,17131,120344
4,기획조정실,868,6337,101052048.0,15946,116419


In [129]:
# 2018
df_divs[3].sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
24,행정국,692,4973,95344355.0,19172,137781
15,시민소통기획관,645,4702,81274853.0,17285,126008
3,기획조정실,561,3867,75619510.0,19555,134794


### 3. 가장 비싼 것을 먹는 부서

In [130]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=False).head(3)
# 시장보다 부시장님들이 더 비싼 것을 먹는다!
# 시장님은 직원들 격려 피자를 자주 사주셔서 평균 금액이 내려간듯...
# 부시장은 1명이라 총금액은 적지만, 비싼 것을 먹는당

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
23,행정1부시장,726,3403,187283110.0,55035,257966
24,행정2부시장,724,4923,231317260.0,46987,319499
20,정무부시장,1534,10530,228080150.0,21660,148683


In [131]:
# 2016년
# 역시 김영란법 시행 전이라 3만원이 넘는다. 
df_divs[1].sort_values(by='인당집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
21,행정1부시장,269,1155,56968960.0,49324,211781
22,행정2부시장,259,1665,75801000.0,45526,292668
19,정무부시장,625,3891,90778450.0,23330,145246


In [132]:
# 2017년
# 라고 생각했는데 시행 후에도 3만원 넘는것 무엇...
df_divs[2].sort_values(by='인당집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
24,행정2부시장,274,1776,87023600.0,49000,317604
23,행정1부시장,309,1486,72742090.0,48952,235411
20,정무부시장,613,4304,86771100.0,20161,141552


In [133]:
# 2018년
# 인당 7만 5천원?? 나중에 부시장 데이터 꼼꼼히 살펴보기
df_divs[3].sort_values(by='인당집행금액', ascending=False).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
22,행정1부시장,148,762,57572060.0,75554,389000
23,행정2부시장,191,1482,68492660.0,46216,358600
19,정무부시장,296,2335,50530600.0,21641,170711


### 4. 가장 싼 것을 먹는 부서

In [134]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=True).head(3)
# 인당 6000원대라니 8배 차이나는 이 빈부격차 무엇..

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
5,기후환경본부,1287,18460,118394360.0,6414,91993
19,재무국,1313,15254,138009766.0,9047,105110
3,기술심사담당관,114,1144,14562200.0,12729,127739


In [136]:
# 2016년
df_divs[1].sort_values(by='인당집행금액', ascending=True).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
14,시민건강국,70,684,7743250.0,11321,110618
18,재무국,162,1787,20495360.0,11469,126515
5,기후환경본부,83,579,6951330.0,12006,83751


In [137]:
# 2017년
df_divs[2].sort_values(by='인당집행금액', ascending=True).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
5,기후환경본부,778,15021,70020101.0,4661,90000
19,재무국,719,8726,75342196.0,8634,104787
3,기술심사담당관,63,690,8544300.0,12383,135624


In [138]:
# 2018년
# 재무국이 꾸준하게 하위권. 시의 예산을 보면 차마 비싼 것을 먹을 수 없는 것일까
df_divs[3].sort_values(by='인당집행금액', ascending=True).head(3)

,1소속부서,cnt,num,집행금액,인당집행금액,회당집행금액
18,재무국,432,4741,42172210.0,8895,97621
6,도시공간개선단,124,952,11298032.0,11868,91113
2,기술심사담당관,39,345,4277500.0,12399,109679
